In [1]:
# !pip install transformers accelerate evaluate datasets peft -q
!pip install ipywidgets sentencepiece protobuf


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
from peft import LoraConfig, get_peft_model
from transformers import LlamaForCausalLM, LlamaTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
from huggingface_hub import login
# login(token=os.getenv("HUGGINGFACE_TOKEN"))

# Load pre-trained LLaMA model
print("Loading LLaMA model...")
model = LlamaForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf")
print("Loading LLaMA tokenizer...")
tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")


Loading LLaMA model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading LLaMA tokenizer...


In [3]:
# Load the training dataset
def preprocess_function(data_record):
    return tokenizer(data_record["title"] + " " + data_record["body"], truncation=True, padding="max_length", max_length=512)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

dataset = load_dataset("json", data_files="../data/training-dataset/pulls.jsonl")
tokenized_datasets = dataset.map(preprocess_function)

In [4]:
# Configure LoRa
config = LoraConfig(
    r=8,  # Rank of the low-rank matrices
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"]  # Apply LoRa to specific layers
)

# Apply LoRa to the model
model = get_peft_model(model, config)

# Set up training
training_args = TrainingArguments(
    output_dir="../models/llama-2-7b",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    num_train_epochs=3
)

trainer = Trainer(model=model, train_dataset=tokenized_datasets["train"], args=training_args)

# Fine-tune the model
trainer.train()

RuntimeError: MPS backend out of memory (MPS allocated: 20.26 GB, other allocations: 464.00 KB, max allowed: 20.40 GB). Tried to allocate 172.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).